In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob, os
import time

O que vou fazer amanhã:
- Abrir todos os bancos do granular activity/opens com o código que gera uma coluna com o nome do arquivo. 
- Ordenar por data e dropar duplicados, assim estimarei a data de envio do email (com a proxy da data da primeira abertura)
- Terei um banco com todas as campanhas com o nome delas (nome do arquivo) no mesmo formato que no aggregated activity
- Depois tenho que juntar todos os bancos do aggregated activity/opened e aggregated activity/not_opened
- Criar uma coluna em cada um desses que especifíque se é de aberto ou fechado
- Em seguida, concatenar opened com not opened
- Mergir com o banco de campanhas
- Ordenar por email e data de envio da campanha(descendente)!
- Daí crio uma contagem que reseta com emails, onde o último email recebido pela pessoa é 1, o segundo 2, assim por diante...
- Depois é apagar com filtros compostos: Se o email 1 (mais recente) é não aberto e o 2, 3,4 e 5

In [20]:
files = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/granular_activity/opens/*.csv')
#df = pd.concat([pd.read_csv(fp, encoding='latin-1',nrows=1 ).assign(New=os.path.basename(fp) ) for fp in files], sort=False)
# Não quero me confiar no nrows, que o pressuposto é que a primeira linha de cada arquivo é o timestamp da primeira abertura
df = pd.concat([pd.read_csv(fp, encoding='latin-1').assign(New=os.path.basename(fp) ) for fp in files], sort=False)
df = df.sort_values(['New', 'Timestamp'])
df = df[['New', 'Timestamp']].drop_duplicates('New')

Existem 6190 arquivos, mas só  4980 estão sendo lidos. Curiosamente, alguns deles são os que não tem. 2 horas pra descobrir que tudo isso era pq são arquivos vazios. A maior parte são testes ab de 100%, então óbvio que ninguém recebe o combo vencedor. Um caso que se eu fosse irresponsável, não ia dar em nada. Pq a perda não existe de vdd, é só a redução natural de categorias, já que algums não tem observações. O código abaixo (a versão limpa, claro), foi pra idenficcar isso.

In [99]:
#paths = pd.DataFrame({'caminhos':files})
#paths['caminhos'] = paths['caminhos'].str[len('C:/Users/ander/Documents/Nossas_mailchimp/granular_activity/opens\\'):] #slicing

#erros = pd.merge(df, paths, left_on='New', right_on='caminhos',how='outer', indicator=True)
#erros[erros['_merge'] != 'both'].caminhos[5000]
#df[df['New'].str.contains('340630_-pdm-solidariedade-nas-ruas-e-ilera-con')]

pd.read_csv('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/opened/308509_-rioacess-vel.csv',
            encoding='latin-1', usecols = [0,1,2,34, 36]).columns

In [22]:
files_opened = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/opened/*.csv')
opened = pd.concat([pd.read_csv(fp, encoding='latin-1',usecols = [0,1,2]
           ).assign(New=os.path.basename(fp) ) for fp in files_opened], sort=False)

In [25]:
opended.shape

(7356289, 9)

In [26]:
files_notopened = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/not_opened/*.csv')
not_opened = pd.concat([pd.read_csv(fp, encoding='latin-1',usecols = [0,1,2]
           ).assign(New=os.path.basename(fp) ) for fp in files_notopened], sort=False)

In [28]:
not_opened.shape

(47942700, 9)

In [29]:
not_opened.to_csv('all_not_opened.csv', index=False)
opended.to_csv('all_opened.csv', index=False)
df.to_csv('all_emails.csv', index=False)

In [39]:
s_nopened = not_opened.sample(frac=0.01)
s_opened = opended.sample(frac=0.02)

In [57]:
s_opened.shape

(147126, 9)

### Dia 2 - Reiniciei o kernel e vou fazer as operações agora sem a memória pesada

open_1 = s_opened[['Email', 'Nome', 'Sobrenome', 'New']]
open_2 = s_opened[['Email Address', 'First Name', 'Last Name', 'New']]
open_3 = s_opened[['E-mail', 'First Name do eleitor', 'New']]
open_3['Sobrenome'] = ''
open_2.columns = ['Email', 'Nome', 'Sobrenome', 'New']
open_3.columns = ['Email', 'Nome', 'New', 'Sobrenome']
open_3 = open_3[['Email', 'Nome', 'Sobrenome', 'New']]
opens = pd.concat([open_1, open_2, open_3])

opens = opens.dropna(subset=['Email'])
opens = opens.merge(df,on='New')
opens['Atividade'] = 'abertura'

n_open_1 = s_nopened[['Email', 'Nome', 'Sobrenome', 'New']]
n_open_2 = s_nopened[['Email Address', 'First Name', 'Last Name', 'New']]
n_open_3 = s_nopened[['E-mail', 'First Name do eleitor', 'New']]
n_open_3['Sobrenome'] = ''
n_open_2.columns = ['Email', 'Nome', 'Sobrenome', 'New']
n_open_3.columns = ['Email', 'Nome', 'New', 'Sobrenome']
n_open_3 = open_3[['Email', 'Nome', 'Sobrenome', 'New']]
n_opens = pd.concat([n_open_1, n_open_2, n_open_3])

n_opens = n_opens.dropna(subset=['Email'])
n_opens = n_opens.merge(df,on='New')
n_opens['Atividade'] = 'não abertura'

In [20]:
start_time = time.time()
#not_opened = pd.read_csv('all_not_opened.csv')
#opened = pd.read_csv('all_opened.csv')
emails = pd.read_csv('all_emails.csv')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.1294398307800293 seconds ---


opens

In [27]:
open_1 = opened[['Email', 'Nome', 'Sobrenome', 'New']]
open_2 = opened[['Email Address', 'First Name', 'Last Name', 'New']]
open_3 = opened[['E-mail', 'First Name do eleitor', 'New']]
open_3['Sobrenome'] = ''
open_2.columns = ['Email', 'Nome', 'Sobrenome', 'New']
open_3.columns = ['Email', 'Nome', 'New', 'Sobrenome']
open_3 = open_3[['Email', 'Nome', 'Sobrenome', 'New']]
opens = pd.concat([open_1, open_2, open_3])

opens = opens.dropna(subset=['Email'])
opens = opens.merge(emails,on='New')
opens['Atividade'] = 'abertura'

C:\Users\ander\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


--- 10.327623844146729 seconds ---


In [92]:
recent_opens = opens.head(10000000)[opens.head(10000000)['Timestamp'] >'2019-01-01 12:12:48']
## por algum motivo, quando eu uso head e não o banco inteiro, vai mais rápido.

In [94]:
recent_opens.to_csv('recent_opens.csv', index=False)

--- 13.886665105819702 seconds ---


In [ ]:
# Já foi rodado
n_opened = not_opened.merge(emails, on='New')

In [107]:
start_time = time.time()

n_opened.to_csv('n_opened.csv', index=False)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1405.8530542850494 seconds ---


Recomeçar daqui: filtrar acima de 2019 e depois salvar (sábado). Na segunda volto e reorganizo, concateno com opens, ordeno e faço os cortes

Estratégia nova: impossível utilizar todas as linhas de não aberturas, então criei uma pasta só pros arquivos a partir de 2019.

Basicamnte recomecei tudo a partir daqui

In [7]:
start_time = time.time()

files_notopened = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/not_opened_recentes/*.csv')
n_opens = pd.concat([pd.read_csv(fp, encoding='latin-1',usecols = [0,1,2]
           ).assign(New=os.path.basename(fp) ) for fp in files_notopened], sort=False)

print("--- %s seconds ---" % (time.time() - start_time))

--- 63.783284187316895 seconds ---


In [8]:
start_time = time.time()

n_open_1 = n_opens[['Email', 'Nome', 'Sobrenome', 'New']].dropna(subset=['Email'])
n_open_2 = n_opens[['Email Address', 'First Name', 'Last Name', 'New']].dropna(subset=['Email Address'])
n_open_2.columns = ['Email', 'Nome', 'Sobrenome', 'New']
n_opens = pd.concat([n_open_1, n_open_2])

#n_opens = n_opens.dropna(subset=['Email'])
n_opens = n_opens.merge(emails,on='New')
n_opens['Atividade'] = 'não abertura'

print("--- %s seconds ---" % (time.time() - start_time))

--- 14.82846474647522 seconds ---


Tática do opens abaixo. Mas pro n_opens, vai ser primeiro merge (já feito), depois corte de datas e só aí reorganizo e concateno. Depois, vou concatenar com o opens, reordenar e fazer os cortes

In [9]:
n_opens.to_csv('recent_n_opens.csv', index=False)

In [10]:
opens = pd.read_csv('recent_opens.csv')

In [19]:
opens.shape[0] + n_opens.shape[0]

17832731

In [24]:
type(all_activities['Timestamp'])

pandas.core.series.Series

In [31]:
all_activities = pd.concat([opens, n_opens])
all_activities = all_activities.sort_values(['Email','Timestamp'])
all_activities.to_csv('all_recent_activities.csv', index=False)
all_activities.to_json('all_recent_activities.json', index=False)

tirar média e desvio padrão do número de emails de cada um e do % de abertura de cada pessoa. Agora é só lazer.

porra, agora ainda vai ter que juntar com as inscrições de cada pessoa. Se é meu rio, ms, mapa..

## Recomeço aqui

In [2]:
df = pd.read_csv('all_recent_activities.csv')

In [3]:
files_notopened = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/not_opened_recentes/*.csv')
n_opens = pd.concat([pd.read_csv(fp, encoding='latin-1'
           ).assign(New=os.path.basename(fp) ) for fp in files_notopened], sort=False)

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (9,11,14,15,19,20,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (6,9,10,11,15,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (20,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,11,19,20,23,24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (4,5,6,9,10,11,14,15,19,20,21,22,23,24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (20,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, resul

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (24,25,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (9,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (4,5,6,9,10,11,14,15,16,17,19,20,21,22,23,24,25,26,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,6,9,10,11,14,15,20,23,24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,6,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (6,9,10,11,14,15,20,21,22,23,24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (6,10,20,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,6,9,10,11,14,15,19,20,24,25,26,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [4]:
n_opens = n_opens[['Email', 'Nome', 'Sobrenome', 'InscriÃ§Ãµes', 'Interesses', 'Member Rating', 'New', 'Email Address',
       'First Name', 'Last Name']].dropna(subset=['InscriÃ§Ãµes'])

In [5]:
files_notopened = glob.glob('C:/Users/ander/Documents/Nossas_mailchimp/aggregate_activity/opened_recentes/*.csv')
opens = pd.concat([pd.read_csv(fp, encoding='latin-1'
           ).assign(New=os.path.basename(fp) ) for fp in files_notopened], sort=False)

C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,6,9,10,11,14,15,16,20,21,22,23,24,25,26,27,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,6,10,19,20,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\ander\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (5,16,19,23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, res

In [6]:
opens = opens[['Email', 'Nome', 'Sobrenome', 'InscriÃ§Ãµes', 'Interesses', 'Member Rating', 'New', 'Email Address',
       'First Name', 'Last Name']].dropna(subset=['InscriÃ§Ãµes'])

In [15]:
opens.Email.nunique()

332762

In [21]:
n_opens = n_opens[['Email', 'InscriÃ§Ãµes', 'Member Rating', 'New']].drop_duplicates('Email', keep='last')
opens = opens[['Email', 'InscriÃ§Ãµes', 'Member Rating', 'New']].drop_duplicates('Email', keep='last')
inscricoes = pd.concat([n_opens, opens])

In [27]:
inscricoes.shape

(1034590, 4)

In [29]:
inscricoes = inscricoes.merge(emails, on='New')

In [32]:
inscricoes = inscricoes.sort_values(['Email', 'Timestamp'])
inscricoes = inscricoes.drop_duplicates("Email")

In [38]:
inscricoes.columns = ['Email', 'Inscrições', 'Menber Rating', 'New', 'Timestamp']

In [40]:
df = df.merge(inscricoes[['Email', 'Inscrições', 'Menber Rating']], on='Email', how='outer', indicator=True)

In [43]:
df.to_csv('all_recent_activities_inscricoes.csv', index=False)

In [2]:
df = pd.read_csv('all_recent_activities_inscricoes.csv')
df = df.drop('_merge', axis=1)

Criar banco no nível do usuário

In [34]:
user_nopen = df[df['Atividade'] == 'não abertura'].groupby('Email', as_index=False).agg({'Atividade': "count"})
user_open = df[df['Atividade'] == 'abertura'].groupby('Email', as_index=False).agg({'Atividade': "count"})
user_geral = df.groupby('Email', as_index=False).agg({"Timestamp":"first","New" : "first" ,"Inscrições":"last","Menber Rating":"last"})
user_nopen.columns = ['Email','n_open']
user_open.columns = ['Email', 'open']
user_geral.columns = ['Email', 'First Email', 'New', 'Inscrições', 'Member Rating']
user = pd.merge(user_nopen, user_open, on='Email', how='outer', indicator=True)
user = user.merge(user_geral, on='Email')

In [44]:
#Taxa de abertura Geral
user.open.sum() /(user.open.sum() + user.n_open.sum()) 

0.14617054448923164

In [78]:
user['Inscrições'] = user['Inscrições'].fillna('0')
user['corte'] = np.where((user['Inscrições'].str.contains('Meu Recife') |user['Inscrições'].str.contains('Minha Jampa') 
         | user['Inscrições'].str.contains('Minha Campinas') | user['Inscrições'].str.contains('Minha Porto Alegre'))
         , "imune", "elegível")

In [81]:
imunes = user[user['corte'] == 'imune']
user = user[user['corte'] == 'elegível']

In [80]:
#user[user['Email'] =='enrica@nossas.org']

In [110]:
# Primeiras exclusões
nunca_abriu = user[user['_merge'] =='left_only']
apagar_1 = nunca_abriu[nunca_abriu['n_open'] >= 3] # quem nunca abriu mesmo já recebendo mais de 3 emails
# nunca abriu, recebeu menos que 2, mas é antigo
apagar_2 = nunca_abriu[(nunca_abriu['n_open'] < 3) & (nunca_abriu['First Email'] < '2019-07-01 00:00:01')]
# pessoas que não abriram nenhum email, mas receberam 1 ou 2 e entraram há menos de 1 ano na base
alerta = nunca_abriu[(nunca_abriu['n_open'] < 3) & (nunca_abriu['First Email'] > '2019-07-01 00:00:01')]

In [208]:
apagar_1.to_csv('nunca_abriu_1.csv')
apagar_2.to_csv('nunca_abriu_2.csv')
alerta.to_csv('nunca_abriu_alerta.csv')

In [3]:
import pandas as pd
df = pd.read_csv('nunca_abriu_1.csv')

In [6]:
df['Member Rating'].value_counts(dropna=False)

NaN    394245
2.0    138078
1.0      6853
3.0      1222
4.0      1120
5.0       270
Name: Member Rating, dtype: int64

In [204]:
apagar_1.shape
apagar_2.shape
alerta.shape

(139194, 9)

In [116]:
nunca_abriu.n_open.sum()

5276450.0

In [119]:
df = df.merge(nunca_abriu[['Email']], on='Email', how='outer', indicator=True)

In [122]:
df = df[df['_merge'] != "both"]

In [143]:
df['corte'] = np.where((df['Inscrições'].str.contains('Meu Recife') |df['Inscrições'].str.contains('Minha Jampa') 
         | df['Inscrições'].str.contains('Minha Campinas') | df['Inscrições'].str.contains('Minha Porto Alegre'))
         , "imune", "elegível")
df = df[df['corte'] == 'elegível']

CARALHO EU SEMPRE QUIS ESSE CÓDIGO (HACK PRA DROPAR DUPLICADOS MANTENDO N LINHAS)

In [146]:
df =df.sort_values(['Email', 'Timestamp'])

In [179]:
df['Inscrições'] = df['Inscrições'].fillna('0')

In [180]:
df_3 = df.groupby('Email').tail(3) #last 3 rows
df_5 = df.groupby('Email').tail(5) #last 5 rows
df_10 = df.groupby('Email').tail(10) #last 10 rows

In [181]:
df_3['abertura'] = np.where((df_3['Atividade'] =='abertura') ,1, 0)
df_3['não abertura'] = np.where((df_3['Atividade'] == 'não abertura'), 1, 0)

df_5['abertura'] = np.where((df_5['Atividade'] =='abertura') ,1, 0)
df_5['não abertura'] = np.where((df_5['Atividade'] == 'não abertura'), 1, 0)

df_10['abertura'] = np.where((df_10['Atividade'] =='abertura') ,1, 0)
df_10['não abertura'] = np.where((df_10['Atividade'] == 'não abertura'), 1, 0)

C:\Users\ander\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ander\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ander\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [182]:
df_3 = df_3.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})
df_5 = df_5.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})
df_10=df_10.groupby(['Email', 'Inscrições'],as_index=False).agg({'Atividade': "count", 'abertura': 'sum', 'não abertura':'sum'})

In [187]:
apagar_3 = df_3[(df_3['Atividade'] == 3) & (df_3['não abertura'] == 3)]
apagar_5 = df_5[(df_5['Atividade'] == 5) & (df_5['não abertura'] == 5)]
apagar_10 = df_10[(df_10['Atividade'] == 10) & (df_10['não abertura'] == 10)]

In [209]:
apagar_3.to_csv('apagar_3.csv', index=False)
apagar_5.to_csv('apagar_5.csv', index=False)
apagar_10.to_csv('apagar_10.csv', index=False)